### 연관 분석(Association Analysis/Rule)
* 데이터 집합으로부터 특정 규칙을 찾아내는 분석
* 예) 장바구니 분석

### 구매내역 예시
1. 기저귀, 맥주, 속옷, 휴지
2. 맥주, 땅콩, 오징어
3. 기저귀, 티셔츠, 수건
4. 맥주, 오징어, 휴지
5. 기저귀, 맥주
6. 기저귀, 휴지
7. 기저귀, 맥주, 속옷
8. 휴지, 속옷
9. 기저귀, 맥주, 수건
10. 수건, 휴지

### 연관규칙 지표
* 지지도(Support) : support A = P(A) :  기저귀의 지지도 = 6/10 (0.6)
    support A->B : P(A∩B) : 기저귀와 맥주를 동시에 구매한 수 / 전체 수 = 4/10 (0.4)
* 신뢰도(Confidence) : confidence A->B : P(A∩B)/P(A) : A의 거래 중 B거래가 포함된 비율, confidence 기저귀->맥주 : 0.4/0.6 = 0.666... = 지지도/P(A)
* 향상도(Lift) : lift A->B : P(A∩B)/P(A)*P(B) = 신뢰도/P(B) = 0.666/0.6 (1.111...)  
    향상도가 1보다 크면 좋다(양의 상관관계)

In [1]:
import pymysql
con = pymysql.connect(host='localhost', user='root', password='root1234', db='project')
cur = con.cursor()
cur.execute('SELECT * FROM article')
data = cur.fetchall()
cur.close()
con.close()

In [2]:
data

(('\t\t\t[이슈 키워드④] 유연석 현봉식 OO\t\t',
  "\t\t\t\t\t\t\t\t연예가에서 가장 핫 했던 키워드의 궁금증을 '독특한 연예뉴스'에서 파헤쳐봤다.\xa0배우라는 직업 외에 유연석과 현봉식은 또 하나의 공통점이 있다. 바로 84년생 쥐띠로 동갑이라는 것.\xa0두 사람은 '수리남'을 통해 함께 호흡을 맞춘 바 있다. 재미있는 사실은 유연석은 실제보다 어려 보이는 외모지만 한봉식은 반대로 성숙한 비주얼을 자랑한다는 점이다.\xa0서로 다른 매력을 지닌 두 배우가 다시 한번 멋진 작품을 통해 조우하길 기대해본다."),
 ('\t\t\t‘빅마우스→작은아씨들’ 대사 논란, 더 이상 내수용 아니라[TV와치]\t\t',
  '이미지 원본보기[뉴스엔 이민지 기자] tvN 토일드라마 \'작은 아씨들\' 대사가 베트남 시청자들의 항의를 받았다. 최근 베트남 Toquoc 등 현지매체에 따르면 베트남 당국은 넷플릭스에서 방영 중인 \'작은 아씨들\'에 대해 자국 내 방영 중단을 요청했다. 베트남 측은 \'작은 아씨들\'이 베트남 전쟁을 실제와 다르게 왜곡했다고 주장했다. 극중 베트남 전쟁에 참전한 군인이자 사조직 정란회를 세운 원기선 장군이 베트남 전쟁에서 무공을 세운 곳으로 묘사한 것과 한 참전군인이 "한국군 한명이 베트콩 스무명을 죽였다"고 말한 장면 등이 문제로 지적됐다. 한국 시청자 입장에서는 빌런처럼 그려지는 인물의 추악한 면모 중 하나일 뿐이라 생각하고 넘어갈 수 있는 장면이지만 베트남 시청자들은 불쾌감을 드러냈다. 최근 종영한 MBC 드라마 \'빅마우스\'는 주인공이 사형수에게 시비를 걸며 "네 엄마가 너 낳고 미역국은 드셨냐. 너 같은 사이코를 낳고 도대체 뭘 드셨냐. �c얌꿍? 아니면 선짓국 같은거?"라고 말하는 장면이 등장했다.이 역시 한국 시청자 입장에서 크게 생각하지 않고 넘어간 장면이었으나 이후 태국 시청자들의 원성을 받을 수 밖에 없었다. 타국을 대표하는 음식을 마치 태아가 잘못된 원인인 것처럼 묘사했다는 것. 넷플릭스 시리즈

In [3]:
# 전처리 함수
def preprocessing(txt):
    import re
    for i, t in enumerate(txt):
        txt[i] = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', t)
    sw = set()
    with open('stopwords-ko.txt', encoding='utf-8') as f:
        for word in f:
            sw.add(word.replace('\n', ''))
    tokens = []
    corpus = []
    from eunjeon import Mecab
    from tqdm import tqdm_notebook
    mecab = Mecab()
    for i in tqdm_notebook(range(len(txt))):
        for t in mecab.morphs(txt[i]):
            if t not in sw and len(t) > 1:
                tokens.append(t)    
        corpus.append(tokens)
    return corpus

In [4]:
contents = []
for row in data:
    contents.append(row[1])

In [5]:
contents[:5]

["\t\t\t\t\t\t\t\t연예가에서 가장 핫 했던 키워드의 궁금증을 '독특한 연예뉴스'에서 파헤쳐봤다.\xa0배우라는 직업 외에 유연석과 현봉식은 또 하나의 공통점이 있다. 바로 84년생 쥐띠로 동갑이라는 것.\xa0두 사람은 '수리남'을 통해 함께 호흡을 맞춘 바 있다. 재미있는 사실은 유연석은 실제보다 어려 보이는 외모지만 한봉식은 반대로 성숙한 비주얼을 자랑한다는 점이다.\xa0서로 다른 매력을 지닌 두 배우가 다시 한번 멋진 작품을 통해 조우하길 기대해본다.",
 '이미지 원본보기[뉴스엔 이민지 기자] tvN 토일드라마 \'작은 아씨들\' 대사가 베트남 시청자들의 항의를 받았다. 최근 베트남 Toquoc 등 현지매체에 따르면 베트남 당국은 넷플릭스에서 방영 중인 \'작은 아씨들\'에 대해 자국 내 방영 중단을 요청했다. 베트남 측은 \'작은 아씨들\'이 베트남 전쟁을 실제와 다르게 왜곡했다고 주장했다. 극중 베트남 전쟁에 참전한 군인이자 사조직 정란회를 세운 원기선 장군이 베트남 전쟁에서 무공을 세운 곳으로 묘사한 것과 한 참전군인이 "한국군 한명이 베트콩 스무명을 죽였다"고 말한 장면 등이 문제로 지적됐다. 한국 시청자 입장에서는 빌런처럼 그려지는 인물의 추악한 면모 중 하나일 뿐이라 생각하고 넘어갈 수 있는 장면이지만 베트남 시청자들은 불쾌감을 드러냈다. 최근 종영한 MBC 드라마 \'빅마우스\'는 주인공이 사형수에게 시비를 걸며 "네 엄마가 너 낳고 미역국은 드셨냐. 너 같은 사이코를 낳고 도대체 뭘 드셨냐. �c얌꿍? 아니면 선짓국 같은거?"라고 말하는 장면이 등장했다.이 역시 한국 시청자 입장에서 크게 생각하지 않고 넘어간 장면이었으나 이후 태국 시청자들의 원성을 받을 수 밖에 없었다. 타국을 대표하는 음식을 마치 태아가 잘못된 원인인 것처럼 묘사했다는 것. 넷플릭스 시리즈 \'수리남\'은 수리남에서 활동한 한국인 마약왕의 실화를 기반으로 했음에도 수리남 정부의 항의를 받은 바 있다. \'K콘텐츠\' 명명된 한국 창작물들은 더이상 한국 

In [8]:
result_txt = preprocessing(contents)

C:\Users\tjoeun-jr-902-02\AppData\Local\Temp\ipykernel_14796\3519052490.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(txt))):


  0%|          | 0/357 [00:00<?, ?it/s]

In [9]:
result_txt[0]

['연예',
 '가장',
 '키워드',
 '궁금증',
 '독특',
 '연예뉴스',
 '파헤쳐',
 '배우',
 '라는',
 '직업',
 '유연석',
 '봉식',
 '공통점',
 '쥐띠',
 '동갑',
 '라는',
 '사람',
 '수리남',
 '통해',
 '호흡',
 '맞춘',
 '재미있',
 '사실',
 '유연석',
 '실제',
 '보다',
 '어려',
 '보이',
 '외모',
 '한봉식',
 '반대',
 '성숙',
 '비주얼',
 '자랑',
 '한다는',
 '서로',
 '매력',
 '지닌',
 '배우',
 '다시',
 '멋진',
 '작품',
 '통해',
 '조우',
 '기대',
 '해본다',
 '이미지',
 '원본',
 '뉴스',
 '이민지',
 '기자',
 '토일',
 '드라마',
 '아씨',
 '대사',
 '베트남',
 '시청자',
 '항의',
 '최근',
 '베트남',
 '현지',
 '매체',
 '따르',
 '베트남',
 '당국',
 '넷플릭스',
 '방영',
 '아씨',
 '대해',
 '자국',
 '방영',
 '중단',
 '요청',
 '베트남',
 '아씨',
 '베트남',
 '전쟁',
 '실제',
 '다르',
 '왜곡',
 '다고',
 '주장',
 '베트남',
 '전쟁',
 '참전',
 '군인',
 '사조직',
 '정란',
 '세운',
 '원기선',
 '장군',
 '베트남',
 '전쟁',
 '무공',
 '세운',
 '묘사',
 '참전',
 '군인',
 '한국',
 '베트콩',
 '스무',
 '죽였',
 '다고',
 '장면',
 '문제',
 '지적',
 '한국',
 '시청자',
 '입장',
 '처럼',
 '그려지',
 '인물',
 '추악',
 '면모',
 '생각',
 '넘어갈',
 '장면',
 '베트남',
 '시청자',
 '불쾌감',
 '드러냈',
 '최근',
 '종영',
 '드라마',
 '마우스',
 '주인공',
 '사형수',
 '시비',
 '엄마',
 '미역국',
 '드셨',
 '사이코',
 '도대체',
 '드셨',
 '선짓국',
 '라고',
 '장

### 피클로 저장

In [11]:
import pickle
with open('result_txt.pkl', 'wb') as f:
    pickle.dump(result_txt, f)

In [21]:
del result_txt

In [1]:
# 피클 불러오기
import pickle
with open('result_txt.pkl', 'rb') as f:
    result_txt = pickle.load(f)

In [21]:
len(result_txt)

357

In [24]:
!pip install apyori

  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5974 sha256=32c77fcbfb61bc90e6654b49463d093d1dde3c054c62c59956289380d8229208
  Stored in directory: c:\users\tjoeun-jr-902-02\appdata\local\pip\cache\wheels\32\2a\54\10c595515f385f3726642b10c60bf788029e8f3a1323e3913a
Successfully built apyori


In [1]:
from apyori import apriori

In [23]:
# 최소지지도, 최소신뢰도, 최소향상도, 최대길이 규칙(룰) 적용
apr = apriori(result_txt[:2], min_support=0.7, min_confidence=0.7, min_lift=1.0, max_length=2)

In [18]:
# result = list(apr)  # 시간이 너무 오래 걸림 -> 버그가 생기지 않았을까 여겨짐

In [25]:
for r in apr:
    print(r)
    print('#####')

RelationRecord(items=frozenset({'ㅇ드렸다'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'ㅇ드렸다'}), confidence=1.0, lift=1.0)])
#####
RelationRecord(items=frozenset({'ㅋㅋㅋ'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'ㅋㅋㅋ'}), confidence=1.0, lift=1.0)])
#####
RelationRecord(items=frozenset({'ㅔㅆ다'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'ㅔㅆ다'}), confidence=1.0, lift=1.0)])
#####
RelationRecord(items=frozenset({'ㅠㅠ류지윤'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'ㅠㅠ류지윤'}), confidence=1.0, lift=1.0)])
#####
RelationRecord(items=frozenset({'가가연'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'가가연'}), confidence=1.0, lift=1.0)])
#####
RelationRecord(items=frozenset({'가감'}), support=1.0, ordered_statistics=[OrderedStatistic(items_bas

RelationRecord(items=frozenset({'다기'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'다기'}), confidence=1.0, lift=1.0)])
#####
RelationRecord(items=frozenset({'다나'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'다나'}), confidence=1.0, lift=1.0)])
#####
RelationRecord(items=frozenset({'다나쿤'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'다나쿤'}), confidence=1.0, lift=1.0)])
#####
RelationRecord(items=frozenset({'다네요'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'다네요'}), confidence=1.0, lift=1.0)])
#####
RelationRecord(items=frozenset({'다녀'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'다녀'}), confidence=1.0, lift=1.0)])
#####
RelationRecord(items=frozenset({'다녀오'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset

#####
RelationRecord(items=frozenset({'밤마실'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'밤마실'}), confidence=1.0, lift=1.0)])
#####
RelationRecord(items=frozenset({'밤새'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'밤새'}), confidence=1.0, lift=1.0)])
#####
RelationRecord(items=frozenset({'밤업소'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'밤업소'}), confidence=1.0, lift=1.0)])
#####
RelationRecord(items=frozenset({'밤잠'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'밤잠'}), confidence=1.0, lift=1.0)])
#####
RelationRecord(items=frozenset({'밤하늘'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'밤하늘'}), confidence=1.0, lift=1.0)])
#####
RelationRecord(items=frozenset({'밥상'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=fr

#####
RelationRecord(items=frozenset({'아메리카노'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'아메리카노'}), confidence=1.0, lift=1.0)])
#####
RelationRecord(items=frozenset({'아메리칸'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'아메리칸'}), confidence=1.0, lift=1.0)])
#####
RelationRecord(items=frozenset({'아무'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'아무'}), confidence=1.0, lift=1.0)])
#####
RelationRecord(items=frozenset({'아무것'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'아무것'}), confidence=1.0, lift=1.0)])
#####
RelationRecord(items=frozenset({'아무래도'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'아무래도'}), confidence=1.0, lift=1.0)])
#####
RelationRecord(items=frozenset({'아무런'}), support=1.0, ordered_statistics=[OrderedStatistic(it

RelationRecord(items=frozenset({'잡혀가'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'잡혀가'}), confidence=1.0, lift=1.0)])
#####
RelationRecord(items=frozenset({'잡혔으니'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'잡혔으니'}), confidence=1.0, lift=1.0)])
#####
RelationRecord(items=frozenset({'잡히'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'잡히'}), confidence=1.0, lift=1.0)])
#####
RelationRecord(items=frozenset({'잡힐'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'잡힐'}), confidence=1.0, lift=1.0)])
#####
RelationRecord(items=frozenset({'장가'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'장가'}), confidence=1.0, lift=1.0)])
#####
RelationRecord(items=frozenset({'장관'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozense

RelationRecord(items=frozenset({'프랑스'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'프랑스'}), confidence=1.0, lift=1.0)])
#####
RelationRecord(items=frozenset({'프러포즈'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'프러포즈'}), confidence=1.0, lift=1.0)])
#####
RelationRecord(items=frozenset({'프레임'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'프레임'}), confidence=1.0, lift=1.0)])
#####
RelationRecord(items=frozenset({'프레젠테이션'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'프레젠테이션'}), confidence=1.0, lift=1.0)])
#####
RelationRecord(items=frozenset({'프렌즈'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'프렌즈'}), confidence=1.0, lift=1.0)])
#####
RelationRecord(items=frozenset({'프로'}), support=1.0, ordered_statistics=[OrderedStatistic(items_b

KeyboardInterrupt: 

In [3]:
result_txt[0]

['연예',
 '가장',
 '키워드',
 '궁금증',
 '독특',
 '연예뉴스',
 '파헤쳐',
 '배우',
 '라는',
 '직업',
 '유연석',
 '봉식',
 '공통점',
 '쥐띠',
 '동갑',
 '라는',
 '사람',
 '수리남',
 '통해',
 '호흡',
 '맞춘',
 '재미있',
 '사실',
 '유연석',
 '실제',
 '보다',
 '어려',
 '보이',
 '외모',
 '한봉식',
 '반대',
 '성숙',
 '비주얼',
 '자랑',
 '한다는',
 '서로',
 '매력',
 '지닌',
 '배우',
 '다시',
 '멋진',
 '작품',
 '통해',
 '조우',
 '기대',
 '해본다',
 '이미지',
 '원본',
 '뉴스',
 '이민지',
 '기자',
 '토일',
 '드라마',
 '아씨',
 '대사',
 '베트남',
 '시청자',
 '항의',
 '최근',
 '베트남',
 '현지',
 '매체',
 '따르',
 '베트남',
 '당국',
 '넷플릭스',
 '방영',
 '아씨',
 '대해',
 '자국',
 '방영',
 '중단',
 '요청',
 '베트남',
 '아씨',
 '베트남',
 '전쟁',
 '실제',
 '다르',
 '왜곡',
 '다고',
 '주장',
 '베트남',
 '전쟁',
 '참전',
 '군인',
 '사조직',
 '정란',
 '세운',
 '원기선',
 '장군',
 '베트남',
 '전쟁',
 '무공',
 '세운',
 '묘사',
 '참전',
 '군인',
 '한국',
 '베트콩',
 '스무',
 '죽였',
 '다고',
 '장면',
 '문제',
 '지적',
 '한국',
 '시청자',
 '입장',
 '처럼',
 '그려지',
 '인물',
 '추악',
 '면모',
 '생각',
 '넘어갈',
 '장면',
 '베트남',
 '시청자',
 '불쾌감',
 '드러냈',
 '최근',
 '종영',
 '드라마',
 '마우스',
 '주인공',
 '사형수',
 '시비',
 '엄마',
 '미역국',
 '드셨',
 '사이코',
 '도대체',
 '드셨',
 '선짓국',
 '라고',
 '장

In [10]:
aa = [['기저귀', '맥주', '속옷', '휴지'],
['맥주', '땅콩', '오징어'],
['기저귀', '티셔츠', '수건'],
['맥주', '오징어', '휴지'],
['기저귀', '맥주'],
['기저귀', '휴지'],
['기저귀', '맥주', '속옷'],
['휴지', '속옷'],
['기저귀', '맥주', '수건'],
['수건', '휴지']]
apr = apriori(aa, min_support=0.2, min_confidence=0.2, min_lift=1.0, max_length=2)

In [11]:
list(apr)

[RelationRecord(items=frozenset({'기저귀'}), support=0.6, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'기저귀'}), confidence=0.6, lift=1.0)]),
 RelationRecord(items=frozenset({'맥주'}), support=0.6, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'맥주'}), confidence=0.6, lift=1.0)]),
 RelationRecord(items=frozenset({'속옷'}), support=0.3, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'속옷'}), confidence=0.3, lift=1.0)]),
 RelationRecord(items=frozenset({'수건'}), support=0.3, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'수건'}), confidence=0.3, lift=1.0)]),
 RelationRecord(items=frozenset({'오징어'}), support=0.2, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'오징어'}), confidence=0.2, lift=1.0)]),
 RelationRecord(items=frozenset({'휴지'}), support=0.5, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozen